In [1]:
import json
import pandas as pd
from cobra.io import read_sbml_model
from mewpy import get_simulator
from mewpy.optimization import BPCY, WYIELD, EA
from mewpy.problems import GOUProblem, GKOProblem
import platform
from cobra.flux_analysis import pfba
from mewpy.optimization.evaluation import BPCY_FVA
import os
os.chdir("optimization")

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'optimization'

In [5]:
model = read_sbml_model(rf"iJN678.xml")
model.objective = "Ec_biomass_SynMixo"
model.solver = "cplex"
model.exchanges.EX_photon_e.bounds = (-30, -30)
model.exchanges.EX_co2_e.bounds = (-2.7, -2.7)
model.exchanges.EX_hco3_e.bounds = (-1.1, 1000)

# model.reactions.EX_gln_DASH_L_e.bounds = (-0.5, -0.5)
# model.reactions.EX_o2_e.bounds = (-0.5, 1000)
sol = model.optimize()

# pfba_sol = pfba(model)
# 
print(model.summary(sol))

Objective
1.0 Ec_biomass_SynMixo = 0.06651314694830654

Uptake
------
  Metabolite        Reaction      Flux  C-Number C-Flux
       ca2_e        EX_ca2_e 0.0003001         0  0.00%
       co2_e        EX_co2_e       2.7         1 34.61%
   cobalt2_e    EX_cobalt2_e 0.0002163         0  0.00%
       cu2_e        EX_cu2_e 0.0002001         0  0.00%
       fe2_e        EX_fe2_e 0.0004979         0  0.00%
       fe3_e        EX_fe3_e 0.0004534         0  0.00%
glc_DASH_D_e EX_glc_DASH_D_e      0.85         6 65.38%
         k_e          EX_k_e   0.01125         0  0.00%
       mg2_e        EX_mg2_e  0.001913         0  0.00%
       mn2_e        EX_mn2_e 0.0002015         0  0.00%
      mobd_e       EX_mobd_e 0.0002015         0  0.00%
       na1_e        EX_na1_e 0.0002519         0  0.00%
       no3_e        EX_no3_e    0.5996         0  0.00%
    photon_e     EX_photon_e        30         0  0.00%
        pi_e         EX_pi_e   0.04591         0  0.00%
       so4_e        EX_so4_e   0.0

In [6]:
with model as temp_model:
    BIOMASS_ID = "Ec_biomass_SynMixo"
    PRODUCT_ID = "EX_fum_e"
    evaluator_2 = BPCY_FVA(BIOMASS_ID, PRODUCT_ID, method="lMOMA")
    simul = get_simulator(temp_model)
    essential_genes = simul.essential_genes()
    problem = GOUProblem(simul, fevaluation=[
                                             evaluator_2], candidate_max_size=5, non_target=essential_genes)
    ea = EA(problem, max_generations=100, visualizer=True)
    final_pop = ea.run()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 678/678 [00:03<00:00, 219.29it/s]
2023-11-14 10:56:30,202	INFO worker.py:1507 -- Calling ray.init() again after it has already been called.


Running GA
Using 32 workers.
Evaluations: 100	Fitness: 0.03162072789745394
Evaluations: 200	Fitness: 0.03162072789745515
Evaluations: 300	Fitness: 0.03162072789745515
Evaluations: 400	Fitness: 0.03162072789745515
Evaluations: 500	Fitness: 0.03162072789745515
Evaluations: 600	Fitness: 0.03162072789745515
Evaluations: 700	Fitness: 0.03162072789745515
Evaluations: 800	Fitness: 0.03162072789747249
Evaluations: 900	Fitness: 0.03162072789747543
Evaluations: 1000	Fitness: 0.03977766899491291
Evaluations: 1100	Fitness: 0.03977766899491301
Evaluations: 1200	Fitness: 0.03977766899491301
Evaluations: 1300	Fitness: 0.03977766899491301
Evaluations: 1400	Fitness: 0.03977766899491301
Evaluations: 1500	Fitness: 0.03977766899491301
Evaluations: 1600	Fitness: 0.03977766899491301
Evaluations: 1700	Fitness: 0.03977766899491301
Evaluations: 1800	Fitness: 0.03977766899491301
Evaluations: 1900	Fitness: 0.03977766899491301
Evaluations: 2000	Fitness: 0.03977766899491301
Evaluations: 2100	Fitness: 0.03977766899

In [7]:
final_pop

[[0.5391444393825989];{'slr0018': 4},
 [0.5366446430714165];{'slr0041': 0, 'slr0018': 4},
 [0.5366446430708771];{'slr0033': 2, 'slr0041': 0, 'slr0018': 4, 'slr0783': 2},
 [0.5366446430708771];{'slr0033': 2, 'slr0040': 0, 'slr0018': 4, 'slr0783': 2},
 [0.5366446430708771];{'slr0033': 2, 'slr0040': 0, 'slr0041': 0, 'slr0018': 4},
 [0.5366446430708771];{'slr0040': 0, 'slr0041': 0, 'slr0018': 4, 'slr0783': 2},
 [0.5100977928334979];{'slr0783': 4, 'slr0018': 4}]

In [39]:
# df = ea.dataframe()
# df.to_csv(rf"synecho_gou_mal.tsv", index=False, sep="\t")
solution = final_pop[0]
sim = problem.simulator
res = sim.simulate(constraints=solution.constraints, method='FBA')
print(res)
# with open(rf"synecho_gou_mal.txt", "w") as f:
#     f.write(res)

objective: 0.01
Status: OPTIMAL
Method:FBA


In [53]:
with model as fucking_model:
    for reaction_id, value  in solution.constraints.items():
        model.reactions.get_by_id(reaction_id).bounds = value
    sol = fucking_model.optimize()
    print(sol[list(solution.constraints.keys())])
    print(fucking_model.summary())

ACBIPGT    0.000001
OXGDC2     0.000009
UAGAAT     0.000000
UAMAS      0.000000
SHCHCS2    0.000009
Name: fluxes, dtype: float64
Objective
1.0 Ec_biomass_SynMixo = 0.01

Uptake
------
  Metabolite        Reaction      Flux  C-Number C-Flux
       ca2_e        EX_ca2_e 4.512E-05         0  0.00%
       co2_e        EX_co2_e       2.7         1 30.34%
   cobalt2_e    EX_cobalt2_e 3.149E-05         0  0.00%
       cu2_e        EX_cu2_e 3.008E-05         0  0.00%
       fe2_e        EX_fe2_e 7.485E-05         0  0.00%
       fe3_e        EX_fe3_e 6.816E-05         0  0.00%
glc_DASH_D_e EX_glc_DASH_D_e      0.85         6 57.30%
      hco3_e       EX_hco3_e       1.1         1 12.36%
         k_e          EX_k_e  0.001692         0  0.00%
       mg2_e        EX_mg2_e 0.0002876         0  0.00%
       mn2_e        EX_mn2_e 3.029E-05         0  0.00%
      mobd_e       EX_mobd_e 3.029E-05         0  0.00%
       na1_e        EX_na1_e 3.787E-05         0  0.00%
       no3_e        EX_no3_e   0